In [ ]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda
!stat cuda
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%cu

/*
CUDA TSP solver
Tuomas Rintamäki 2016
tuomas.rintamaki@aalto.fi
*/

/*
License for the helper code for reading the TSPLIB files:
Copyright (c) 2014, Texas State University. All rights reserved.
Redistribution and use in source and binary forms, with or without modification,
are permitted for academic, research, experimental, or personal use provided
that the following conditions are met:
   * Redistributions of source code must retain the above copyright notice,
     this list of conditions and the following disclaimer.
   * Redistributions in binary form must reproduce the above copyright notice,
     this list of conditions and the following disclaimer in the documentation
     and/or other materials provided with the distribution.
   * Neither the name of Texas State University nor the names of its
     contributors may be used to endorse or promote products derived from this
     software without specific prior written permission.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR
ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
(INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON
ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
Author: Martin Burtscher
*/

#include <stdlib.h>
#include <stdio.h>
#include <math.h>
#include <limits.h>
#include <sys/time.h>
#include <cuda.h>
#include <curand_kernel.h>

#define dist(a, b) (sqrtf((px[a] - px[b]) * (px[a] - px[b]) + (py[a] - py[b]) * (py[a] - py[b])))
#define swap(a, b) {int tmp = a;  a = b;  b = tmp;}

static __device__ volatile int best_d;
static __device__ volatile int sol_d;

int best_c = 999999999;
int sol_c = 0;

/******************************************************************************/
/*** GPU code **************************************************************/
/******************************************************************************/

__global__ void Init()
{
  sol_d = 0;
  best_d = 999999999;
}

__global__ void GPU_TwoOpt(int cities, float *posx, float *posy, float *px, float *py, int *tour, int *len)
{
  int a,b,c,d, Dab;
  int i,j,mini,minj,from,to,cost,offset;
  int minchange, change;

  offset = blockIdx.x*cities;

  // copy the city coordinates and set the initial tour for each block
  for (i = 0; i < cities; i++) {
    px[offset+i] = posx[i];
    py[offset+i] = posy[i];
    tour[offset+i] = i;
  }
 
  // do serial permutation of the city coordinates so that initial tour is randomized
  curandState rndstate;
  curand_init(blockIdx.x, 0, 0, &rndstate);
  for (i = 0; i < cities; i++) {
    j = curand(&rndstate) % (cities);
    swap(tour[offset+i], tour[offset+j]);
  }
  
  // search for 2-opt moves
  do {
      minchange = 0;
      i = 0;
      b = tour[offset+cities-1];

      while (i < cities-3) {
          a = b;
          i = i+1;
          b = tour[offset+i];
          Dab = dist(a,b);
          j = i+1;
          d = tour[offset+j];

          while (j < cities-1) {
              c = d;
              j = j+1;
              d = tour[offset+j];

              change = dist(a,c) - dist(c,d) + dist(b,d) - Dab;
              if (change < minchange) {
                  minchange = change;
                  mini = i;
                  minj = j;
              }
          }
      }

      // apply the best move
      if (minchange < 0) {
        i = mini;
        j = minj-1;

        while (i < j) {
          swap(tour[offset+j], tour[offset+i]);
          i++;
          j--;
        }
      }
  } while (minchange < 0);

  // we have a local minimum so compute the cost of the tour
  cost = 0;
  
  for (i = 0;i < cities - 1;i++) 
  {
    from = tour[offset+i];
    to = tour[offset+i+1];
    cost += dist(from,to);
  }
  
  // check if the current local minimum is the best solution so far and save it if necessary
  atomicMin((int *)&best_d, cost);
  if (best_d == cost) {
    sol_d = blockIdx.x;
  }
}


/******************************************************************************/
/*** CPU code **************************************************************/
/******************************************************************************/



void CPU_TwoOpt(int cities, int restarts, float *posx, float *posy, float *px, float *py, int *tour, int *len)
{
  int a,b,c,d, Dab;
  int i,j,mini,minj,from,to,cost,offset;
  int minchange, change;

  for (int k = 0; k < restarts; k++){
      
  offset = k*cities;

  // copy the city coordinates and set the initial tour for each block
  for (i = 0; i < cities; i++) {
    px[offset+i] = posx[i];
    py[offset+i] = posy[i];
    tour[offset+i] = i;
  }
 
  // do serial permutation of the city coordinates so that initial tour is randomized
  srand(k);
  for (i = 0; i < cities; i++) {
    j = rand() % (cities);
    swap(tour[offset+i], tour[offset+j]);
  }
  
  // search for 2-opt moves
  do {
      minchange = 0;
      i = 0;
      b = tour[offset+cities-1];

      while (i < cities-3) {
          a = b;
          i = i+1;
          b = tour[offset+i];
          Dab = dist(a,b);
          j = i+1;
          d = tour[offset+j];

          while (j < cities-1) {
              c = d;
              j = j+1;
              d = tour[offset+j];

              change = dist(a,c) - dist(c,d) + dist(b,d) - Dab;
              if (change < minchange) {
                  minchange = change;
                  mini = i;
                  minj = j;
              }
          }
      }

      // apply the best move
      if (minchange < 0) {
        i = mini;
        j = minj-1;

        while (i < j) {
          swap(tour[offset+j], tour[offset+i]);
          i++;
          j--;
        }
      }
  } while (minchange < 0);

  // we have a local minimum so compute the cost of the tour
  cost = 0;
  
  for (i = 0;i < cities - 1;i++) 
  {
    from = tour[offset+i];
    to = tour[offset+i+1];
    cost += dist(from,to);
  }

  // check if the current local minimum is the best solution so far and save it if necessary
  if (best_c > cost) {
    
    best_c = cost;
    sol_c = k;
  }
  }
}



/******************************************************************************/
/*** helper code **************************************************************/
/******************************************************************************/

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

static void CudaTest(char *msg)
{
  cudaError_t e;

  cudaThreadSynchronize();
  if (cudaSuccess != (e = cudaGetLastError())) {
    fprintf(stderr, "%s: %d\n", msg, e);
    fprintf(stderr, "%s\n", cudaGetErrorString(e));
    exit(-1);
  }
}


/******************************************************************************/
/*** read TSPLIB input ********************************************************/
/******************************************************************************/

static int readInput(char *fname, float **posx_d, float **posy_d)  // ATT and CEIL_2D edge weight types are not supported
{
  int ch, cnt, in1, cities;
  float in2, in3;
  FILE *f;
  char str[256];  // potential for buffer overrun

  f = fopen(fname, "rt");
  if (f == NULL) {fprintf(stderr, "could not open file %s\n", fname);  exit(-1);}

  ch = getc(f);  while ((ch != EOF) && (ch != '\n')) ch = getc(f);
  ch = getc(f);  while ((ch != EOF) && (ch != '\n')) ch = getc(f);
  ch = getc(f);  while ((ch != EOF) && (ch != '\n')) ch = getc(f);

  ch = getc(f);  while ((ch != EOF) && (ch != ':')) ch = getc(f);
  fscanf(f, "%s\n", str);
  cities = atoi(str);
  if (cities <= 2) {fprintf(stderr, "only %d cities\n", cities);  exit(-1);}

  cudaMallocManaged(posy_d, sizeof(float) * cities);
  cudaMallocManaged(posx_d, sizeof(float) * cities);

  ch = getc(f);  while ((ch != EOF) && (ch != '\n')) ch = getc(f);
  fscanf(f, "%s\n", str);
  if (strcmp(str, "NODE_COORD_SECTION") != 0) {fprintf(stderr, "wrong file format\n");  exit(-1);}

  cnt = 0;
  while (fscanf(f, "%d %f %f\n", &in1, &in2, &in3)) {
    (*posx_d)[cnt] = in2;
    (*posy_d)[cnt] = in3;
    cnt++;
    if (cnt > cities) {fprintf(stderr, "input too long\n");  exit(-1);}
    if (cnt != in1) {fprintf(stderr, "input line mismatch: expected %d instead of %d\n", cnt, in1);  exit(-1);}
  }
  if (cnt != cities) {fprintf(stderr, "read %d instead of %d cities\n", cnt, cities);  exit(-1);}

  fscanf(f, "%s", str);
  if (strcmp(str, "EOF") != 0) {fprintf(stderr, "didn't see 'EOF' at end of file\n");  exit(-1);}

  fclose(f);
  return cities;
}

int main()
{
  int cities, restarts, climbs, best, sol;
  int *tour;
  int *tour_d, *len_d;
  float *posx_d, *posy_d, *px_d, *py_d;
  clock_t clk;
  cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
 

  restarts = 128;

  
  cities = readInput("/content/qa194.tsp", &posx_d, &posy_d);
  
 
  //allocate memory for saving blockwise x and y positions on the device as well as the tour orders
  
  checkCudaErr(cudaMallocManaged(&px_d, restarts*cities*sizeof(float)), "cudaMallocManaged1");
  checkCudaErr(cudaMallocManaged(&py_d, restarts*cities*sizeof(float)), "cudaMallocManaged2");
  checkCudaErr(cudaMallocManaged(&tour_d, restarts*cities*sizeof(int)), "cudaMallocManaged3");
  
  // also, allocate memory for saving the blockwise tour lengths and the final solution
  
  checkCudaErr(cudaMallocManaged(&len_d, restarts*sizeof(int)), "cudaMallocManaged4");
  checkCudaErr(cudaMalloc((void **)&sol_d, cities*sizeof(int)), "cudaMallocManaged5");
 
  // Running Host Function
  
  clk = clock();

  CPU_TwoOpt(cities, restarts, posx_d, posy_d, px_d, py_d, tour_d, len_d);
  
	clk = clock() - clk;
	double ct = (((double)clk) / CLOCKS_PER_SEC) * 1000;
	printf("Total time on CPU: %f msecs\n", ct);
  
  
  // output results
 
  printf("best instance = %d \n", sol_c);
  printf("best found tour length = %d\n \n", best_c);

  // delete host results
 
  checkCudaErr(cudaFree(px_d), "cudaFree1");
  checkCudaErr(cudaFree(py_d), "cudaFree2");
  checkCudaErr(cudaFree(tour_d), "cudaFree1");
  checkCudaErr(cudaFree(len_d), "cudaFree2");
 

  //again allocate memory for saving blockwise x and y positions on the device as well as the tour orders
  
  checkCudaErr(cudaMallocManaged(&px_d, restarts*cities*sizeof(float)), "cudaMallocManaged1");
  checkCudaErr(cudaMallocManaged(&py_d, restarts*cities*sizeof(float)), "cudaMallocManaged2");
  checkCudaErr(cudaMallocManaged(&tour_d, restarts*cities*sizeof(int)), "cudaMallocManaged3");
  
  // also, again allocate memory for saving the blockwise tour lengths and the final solution
  
  checkCudaErr(cudaMallocManaged(&len_d, restarts*sizeof(int)), "cudaMallocManaged4");

  
  // initialize device variables
 
  Init<<<1, 1>>>();
 
  // running device functions
 
  cudaEventRecord(start);
  GPU_TwoOpt<<<restarts, 1>>>(cities, posx_d, posy_d, px_d, py_d, tour_d, len_d);
  cudaDeviceSynchronize();
  cudaEventRecord(stop);
  CudaTest("kernel launch failed");
  
  float dt = 0;
	cudaEventElapsedTime(&dt, start, stop);
	printf("Total time on GPU: %f msecs\n", dt);

  
  // read results
 
  cudaMemcpyFromSymbol(&best, best_d, sizeof(int));
  cudaMemcpyFromSymbol(&sol, sol_d, sizeof(int));
  tour = (int *)malloc(sizeof(int)*cities);  if (tour == NULL) {fprintf(stderr, "cannot allocate tour\n");  exit(-1);}
  cudaMemcpy(tour, &tour_d[sol*cities], sizeof(int)*cities, cudaMemcpyDeviceToHost);
  

  // output results
 
  printf("best instance = %d \n", sol);
  printf("best found tour length = %d\n", best);
 
  //for (int i = 0; i < cities; i++) {
    //printf("node %d \n", tour[i]);
  //}

  // free all memory
 
  checkCudaErr(cudaFree(posx_d), "cudaFree3");
  checkCudaErr(cudaFree(posy_d), "cudaFree4");
  checkCudaErr(cudaFree(px_d), "cudaFree5");
  checkCudaErr(cudaFree(py_d), "cudaFree6");
  checkCudaErr(cudaFree(tour_d), "cudaFree7");
  checkCudaErr(cudaFree(len_d), "cudaFree8");
 
 
  return 0;
}